<a href="https://colab.research.google.com/github/maxrgnt/pythdc2-project2/blob/master/Clean2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Panel Data
import pandas as pd
# System folders
import os
from pathlib import Path

In [2]:
def abbreviate(stateName):
    abrvDict = {'Alaska':'AL',
                'Arizona':'AZ',
                'California':'CA',
                'Idaho':'ID',
                'Maine':'ME',
                'Michigan':'MI',
                'Minnesota':'MN',
                'Montana':'MT',
                'New Mexico':'NM',
                'New York':'NY',
                'North Dakota':'ND',
                'Texas':'TX',
                'Vermont':'VT',
                'Washington':'WA'}
    abrv = ''
    if stateName in abrvDict:
        abrv = abrvDict[stateName]
    return abrv

def safeDrop(df, cols):
    print(df.shape)
    for col in df.columns:
        if col in cols:
            print(f'removing: {col}')
            df.drop([col], axis=1, inplace=True)
    print(df.shape)
    return df

## Border Data

In [66]:
dataPath = Path.joinpath(Path.cwd(),'sourceData','borderCrossing.csv')
df = pd.read_csv(dataPath)
df.sample(3)

,Port Name,State,Port Code,Border,Date,Measure,Value,Location
46899,Limestone,Maine,118,US-Canada Border,39934,Personal Vehicle Passengers,5390,POINT (-67.79 46.92)
41974,El Paso,Texas,2402,US-Mexico Border,40269,Personal Vehicle Passengers,1575984,POINT (-106.45 31.76)
15777,Port Huron,Michigan,3802,US-Canada Border,42064,Bus Passengers,4483,POINT (-82.42 43)


In [67]:
df['Measure'].value_counts()

Personal Vehicle Passengers    30196
Bus Passengers                 28820
Pedestrians                    28697
Train Passengers               27623
Name: Measure, dtype: int64

In [68]:
# Only interested in Passenger / Pedestrian crossings
people = df['Measure'].str.contains('Passengers|Pedestrians', case = False)

In [69]:
# Check to see how much data frame shrinks after filtering down
print(f'All measures: {df.shape}')
#df = df[people] # This gave index error when re-running, changed to df.loc
df = df.loc[people]
print(f'Just people: {df.shape}')

All measures: (115336, 8)
Just people: (115336, 8)


In [70]:
# Break out Location into latitude and longitude,
                # substring from 'POINT( ' to ')' and split on ' ' grabbing first then second element set as float,
if 'Location' in df.columns:
    df['Latitude'] = df['Location'].str[len('POINT ('):-1].str.split(' ').str[1].astype(float)
    df['Longitude'] = df['Location'].str[len('POINT ('):-1].str.split(' ').str[0].astype(float)

In [71]:
# Drop unnecessary columns
df = safeDrop(df, ['Port Code','Port Name','Location','Unnamed: 0','index'])

(115336, 10)
removing: Port Name
removing: Port Code
removing: Location
(115336, 7)


In [72]:
# Get state abrv
df['Abrv'] = df['State'].apply(abbreviate)

In [73]:
# Handle the Date column
if 'Date' in df.columns:
    df['newDate'] = pd.to_datetime(df['Date'])
    df['Year'] = df['newDate'].dt.year.astype(int)

In [74]:
# drop unneeded rows
# index of all rows where df['Year'] < 1996
print(df.shape)
dropIndex = df.loc[df['Year'].astype(int)>2018].index
df.drop(dropIndex, inplace=True)
print(df.shape)

(115336, 10)
(115336, 10)


In [75]:
# reorganize columns
df = df[['Abrv','State','Longitude','Latitude','Border','Year','Measure','Value']]

In [76]:
df.sample(5)

,Abrv,State,Longitude,Latitude,Border,Year,Measure,Value
95331,ME,Maine,-70.40,45.81,US-Canada Border,1970,Pedestrians,59
108480,WA,Washington,-122.26,49.00,US-Canada Border,1970,Bus Passengers,5288
91495,TX,Texas,-106.45,31.76,US-Mexico Border,1970,Personal Vehicle Passengers,4050300
108929,WA,Washington,-117.83,49.00,US-Canada Border,1970,Bus Passengers,516
57154,VT,Vermont,-72.09,45.01,US-Canada Border,1970,Bus Passengers,6921


In [51]:
# Remove non-pedestrian values to shrink file
# df.to_csv(Path.joinpath(Path.cwd(),'data','borderCrossing.csv'), index = False)

## GDP Data

In [10]:
dataPath = Path.joinpath(Path.cwd(),'sourceData','pctChangeGDP.csv')
df = pd.read_csv(dataPath)
df.sample(3)

,GeoFips,GeoName,1964,1965,1966,1967,1968,1969,1970,1971,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
35,38000,North Dakota,-2.0,14.7,1.2,1.5,4.5,11.2,5.3,14.7,...,0.9,10.7,16.2,24.7,4.9,8.7,-6.2,-7.6,1.7,6.0
44,48000,Texas,7.2,8.0,8.6,8.5,11.5,9.2,7.7,9.6,...,-6.0,6.3,7.6,6.0,6.4,4.2,0.2,-0.2,5.8,7.3
48,53000,Washington,2.0,8.2,15.7,9.7,8.7,6.3,0.1,3.6,...,-0.6,3.4,3.8,5.6,4.6,5.5,6.4,4.8,6.0,7.8


In [11]:
# rename GeoName to State
df.rename(columns={'GeoName':'State'}, inplace=True)

In [12]:
# Get state abrv
df['Abrv'] = df['State'].apply(abbreviate)

In [15]:
# drop unneeded rows\n",
# index of all rows where df['Abrv'] == ''
print(df.shape)
dropIndex = df.loc[df['Abrv']==''].index
df.drop(dropIndex, inplace=True)
print(df.shape)

(14, 58)
(14, 58)


In [16]:
# SAFE DROP
df = safeDrop(df, ['GeoFips'])

(14, 58)
removing: GeoFips
(14, 57)


In [17]:
if 'Year' not in df.columns:
  df = pd.melt(df, id_vars=['State','Abrv'], var_name='Year', value_name = 'Value')

In [21]:
# drop unneeded rows
# index of all rows where df['Year'] < 1996
print(df.shape)
dropIndex = df.loc[df['Year'].astype(int)<1997].index
df.drop(dropIndex, inplace=True)
print(df.shape)

(770, 4)
(308, 4)


In [31]:
df.sample(5)

,FIPS,State,Year,Population,LaborForce,PercentOfPopulation,Employed,PercentOfLaborEmp,Unemployed,PercentOfLaborUnemp,Abrv
1670,30,Montana,2007,750440.0,502070.0,66.9,484189.0,64.5,17881.0,3.6,MT
2228,4,Arizona,2018,5606275.0,3439755.0,61.4,3273550.0,58.4,166205.0,4.8,AZ
1427,53,Washington,2002,4601381.0,3082839.0,67.0,2854762.0,62.0,228077.0,7.4,WA
1379,2,Alaska,2002,452064.0,326401.0,72.2,302580.0,66.9,23821.0,7.3,AL
1308,38,North Dakota,2000,485026.0,342568.0,70.6,332407.0,68.5,10161.0,3.0,ND


In [13]:
# Un-pivoting
# df.to_csv(Path.joinpath(Path.cwd(),'data','pctChangeGDP.csv'), index = False)

## Unemployment Data

In [35]:
dataPath = Path.joinpath(Path.cwd(),'sourceData','unemployment.csv')
df = pd.read_csv(dataPath)
df.sample(3)

,FIPS,Stata,Year,Population,LaborForce,PercentOfPopulation,Employed,PercentOfLaborEmp,Unemployed,PercentOfLaborUnemp
526,53,Washington,1985,3258750.0,2097674.0,64.4,1922407.0,59.0,175267.0,8.4
791,53,Washington,1990,3676359.0,2525326.0,68.7,2392891.0,65.1,132435.0,5.2
119,16,Idaho,1978,631667.0,416943.0,66.0,394249.0,62.4,22694.0,5.4


In [36]:
# rename GeoName to State
df.rename(columns={'Stata':'State'}, inplace=True)

In [37]:
# get abbreviations
df['Abrv'] = df['State'].apply(abbreviate)
# drop unneeded rows
# index of all rows where df['Abrv'] == ''
print(df.shape)
dropIndex = df.loc[df['Abrv']==''].index
df.drop(dropIndex, inplace=True)
print(df.shape)

(2279, 11)
(602, 11)


In [38]:
# drop unneeded rows
# index of all rows where df['Year'] < 1996
print(df.shape)
dropIndex = df.loc[df['Year']<1996].index
df.drop(dropIndex, inplace=True)
print(df.shape)

(602, 11)
(322, 11)


In [39]:
for col in ['Unemployed','Employed','LaborForce','Population']:
    if col in list(df.columns):
        df['UnemploymentRate'] = df['Unemployed'].div(df['Employed'])
        df['LaborRate'] = df['LaborForce'].div(df['Population'])

In [40]:
# drop unneeded columns
df = safeDrop(df, ['FIPS','PercentOfPopulation','PercentOfLaborEmp','PercentOfLaborUnemp','Population','LaborForce','Employed','Unemployed'])

(322, 13)
removing: FIPS
removing: Population
removing: LaborForce
removing: PercentOfPopulation
removing: Employed
removing: PercentOfLaborEmp
removing: Unemployed
removing: PercentOfLaborUnemp
(322, 5)


In [41]:
df.sample(3)

,State,Year,Abrv,UnemploymentRate,LaborRate
1931,Michigan,2012,MI,0.100323,0.599791
1617,Montana,2006,MT,0.036184,0.667396
1750,Alaska,2009,AL,0.083953,0.702570


In [36]:
# Un-pivoting
# df.to_csv(Path.joinpath(Path.cwd(),'data','unemployment.csv'), index = False)